<a href="https://colab.research.google.com/github/No1-Park/AI-introduction-Practices/blob/main/CNN_MNIST_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 사용할 라이브러리 불러 오기

import numpy as np                                         
from keras.utils import np_utils                           # 숫자-> One-hot Vector 를 위한 라이브러리
from keras.datasets import mnist                           # MNISt Dataset을 가져오기 위한 라이브러리
from keras.models import Sequential                        # 순차적인 모델을 만들기 위한 Sequential 함수
from keras.layers import Dense, Activation                 # 필요한 충돌을 가지고 있는 layers 라이브러리
from keras.layers import Conv2D, MaxPooling2D, Flatten     # 
from google.colab.patches import cv2_imshow                # Google Colab에서 이미지를 보기 위한 cv2_imshow함수
import matplotlib.pyplot as plt

In [ ]:
# Keras 라이브러리 자체에서 제공하는 MNIST 데이터셋을 가져온다

(x_train, y_train), (x_test, y_test) = mnist.load_data()

print('x_train :', np.shape(x_train))
print('y_train :', np.shape(y_train))

for i in range(10):
  cv2_imshow(x_test[i])


In [ ]:
x_train = x_train.reshape(60000, 28, 28, 1).astype('float32') / 255.0
x_test = x_test.reshape(10000, 28, 28, 1).astype('float32') / 255.0

In [ ]:
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

print('y_train :', np.shape(y_train))

In [ ]:
x_valid = x_train[50000:]
y_valid = y_train[50000:]
x_train = x_train[:50000]
y_train = y_train[:50000]

print('x_train :', np.shape(x_train))
print('x_valid :', np.shape(x_valid))

In [ ]:
width = 28
height = 28
channel = 1

model = Sequential(name='MNIST_CNN')
model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', 
                 input_shape=(width, height, channel)))
model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(512, activation='relu'))
model.add(Dense(10, activation='softmax'))

model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

hist = model.fit(x_train, y_train,
                 epochs=10,
                 batch_size=32,
                 validation_data=(x_valid, y_valid))

In [ ]:
plt.plot(hist.history['loss'], 'y', label='train loss')
plt.plot(hist.history['val_loss'], 'r', label='val loss')

plt.ylim([0.0, 0.6])

plt.xlabel('epoch')
plt.ylabel('loss')

plt.legend(loc='upper right')

plt.show()

In [ ]:
plt.plot(hist.history['accuracy'], 'b', label='train accuracy')
plt.plot(hist.history['val_accuracy'], 'g', label='val accuracy')

plt.ylim([0.8, 1.0])

plt.xlabel('epoch')
plt.ylabel('accurary')

plt.legend(loc='lower right')

plt.show()

In [ ]:
loss_and_acc = model.evaluate(x_test, y_test, batch_size=32)
print('Test: Loss and Accuracy')
print(loss_and_acc)

In [ ]:
# 모델을 직접 써보자
 # Test Set 20개를 맞춰보자
for i in range(20):
  
  # predict()를 통해 입력을 모델에 통과시킬 수 있다
  # output은 softmax의 출력으로 10개의 값을 갖느다
  output = model.predict(x_test[i].reshape(1,28, 28, 1))
  # predict()의 입력은 맨 앞에 차원을 하나 추가해줘야 한다.
  
  # 이미지를 보기 위해 다시 사각형으로 만들고, 다시 255를 곱해준다.
  cv2_imshow(x_test[i].reshape(28, 28) * 255)
  
  # argmax() 함수는 가장 최댓값을 갖는 Index(번째)를 찾아준다.
  print('예상 숫자 분류 :', np.argmax(output))
  
  # 모델의 출력과 실제 값을 비교 후 다르면 문장을 출력한다.
  if np.argmax(output) != np.argmax(y_test[i]) :
    print('틀렸습니다!')

In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

In [ ]:
model.save('/gdrive/My Drive/Mnist_CNN.h5')

In [ ]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

%matplotlib inline

SVG(model_to_dot(model, show_shapes=True).create(prog='dot', format='svg'))